In [11]:
import numpy as np
import pyomo.environ as pe

In [12]:
model1 = pe.ConcreteModel()
model1.IDXflow = pe.RangeSet(9)
model1.IDXchem = ['A', 'B', 'C']
model1.IDXprocess = [1, 2, 3]
model1.f = pe.Var(model1.IDXflow, within=pe.NonNegativeReals)  # flows
model1.y = pe.Var(model1.IDXprocess, within=pe.Binary)  # process selection

fixed_cost = {1: 10, 2: 15, 3: 20}
variable_cost = {1: 2.5, 2: 4.0, 3: 5.5}
raw_cost = {'A': 5, 'B': 9.5, 'C': 18}
conversion = {1: 0.90, 2: 0.82, 3: 0.95}

model1.fc = pe.Param(model1.IDXprocess, initialize=fixed_cost)
model1.vc = pe.Param(model1.IDXprocess, initialize=variable_cost)
model1.rc = pe.Param(model1.IDXchem, initialize=raw_cost)
model1.x = pe.Param(model1.IDXprocess, initialize=conversion)

In [13]:
def obj_rule(m):
    return m.rc['C'] * m.f[9] \
           - (m.rc['A'] + m.vc[1]) * m.f[1] \
           - m.rc['B'] * m.f[3] \
           - m.vc[2] * m.f[5] \
           - m.vc[3] * m.f[6] \
           - sum(m.fc[i] * m.y[i] for i in m.IDXprocess)  # revenue - raw material cost - variable cost - fixed cost
model1.obj = pe.Objective(rule=obj_rule, sense=pe.maximize)

In [14]:
def max_demand_rule(m):
    return m.f[9] <= 10
model1.max_demand = pe.Constraint(rule=max_demand_rule)  # in case 1


def max_supply_rule(m):
    return m.f[1] <= 16
model1.max_supply = pe.Constraint(rule=max_supply_rule)


# mass balance - chemical process
def mass_balanceI_rule(m):
    return m.f[2] == m.x[1] * m.f[1]
model1.mass_balanceI = pe.Constraint(rule=mass_balanceI_rule)


def mass_balanceII_rule(m):
    return m.f[7] == m.x[2] * m.f[5]
model1.mass_balanceII = pe.Constraint(rule=mass_balanceII_rule)


def mass_balanceIII_rule(m):
    return m.f[8] == m.x[3] * m.f[6]
model1.mass_balanceIII = pe.Constraint(rule=mass_balanceIII_rule)


# mass balance - mix and split
def mass_balanceB1_rule(m):
    return m.f[4] == m.f[5] + m.f[6]
model1.mass_balanceB1 = pe.Constraint(rule=mass_balanceB1_rule)


def mass_balanceB2_rule(m):
    return m.f[4] == m.f[2] + m.f[3]
model1.mass_balanceB2 = pe.Constraint(rule=mass_balanceB2_rule)

def mass_balanceC_rule(m):
    return m.f[9] == m.f[7] + m.f[8]
model1.mass_balanceC = pe.Constraint(rule=mass_balanceC_rule)


# process selection
def upper_bound1_rule(m):
    return m.f[1] <= 16 * m.y[1]
model1.upperbound1 = pe.Constraint(rule=upper_bound1_rule)


def upper_bound2_rule(m):
    return m.f[5] <= 10/m.x[2] * m.y[2]
model1.upperbound2 = pe.Constraint(rule=upper_bound2_rule)


def upper_bound3_rule(m):
    return m.f[6] <= 10/m.x[3] * m.y[3]
model1.upperbound3 = pe.Constraint(rule=upper_bound3_rule)


def exclusive_strategy_rule(m):
    return sum(m.y[i] for i in [2, 3]) == 1
model1.exclusive_strategy = pe.Constraint(rule=exclusive_strategy_rule)  #XOR

In [15]:
# 1.1
solver1 = pe.SolverFactory('gurobi')
result_obj = solver1.solve(model1, tee=True)
model1.pprint()
print(result_obj) # remember to * 100

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-01
Read LP format model from file /var/folders/b0/z7p69zhx459bv344t3ml6by00000gp/T/tmp2pq_xh98.pyomo.lp
Reading time = 0.00 seconds
x13: 13 rows, 13 columns, 26 nonzeros
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[rosetta2])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 13 rows, 13 columns and 26 nonzeros
Model fingerprint: 0xb814a06e
Variable types: 10 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [8e-01, 2e+01]
  Objective range  [4e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 8 rows and 8 columns
Presolve time: 0.00s
Presolved: 5 rows, 5 columns, 11 nonzeros
Variable types: 3 continuous, 2 integer (2 binary)
Found heuristic solution: objective 4.5934959

Root relaxation: objective 5.754386e+00, 2 iterations, 0.00 seconds (0.

In [16]:
# split the original flow 9 into two flows with selling price 1.8 and 1.5
# f9[1] must <= 10, f9[2] must <= 5
model1 = pe.ConcreteModel()
model1.IDXflow = pe.RangeSet(8)
model1.IDXchem = ['A', 'B', 'C1', 'C2']
model1.IDXprocess = [1, 2, 3]
model1.f = pe.Var(model1.IDXflow, within=pe.NonNegativeReals)
model1.y = pe.Var(model1.IDXprocess, within=pe.Binary)
model1.f9 = pe.Var([1, 2], within=pe.NonNegativeReals)

fixed_cost = {1: 10, 2: 15, 3: 20}
variable_cost = {1: 2.5, 2: 4.0, 3: 5.5}
raw_cost = {'A': 5.0, 'B': 9.5, 'C1': 18, 'C2': 15}
conversion = {1: 0.90, 2: 0.82, 3: 0.95}

model1.fc = pe.Param(model1.IDXprocess, initialize=fixed_cost)
model1.vc = pe.Param(model1.IDXprocess, initialize=variable_cost)
model1.rc = pe.Param(model1.IDXchem, initialize=raw_cost)
model1.x = pe.Param(model1.IDXprocess, initialize=conversion)

In [17]:
# because the selling price of f9[1] is higher than that of f9[2]
# it will automatically choose f9[1] to be the maximum before increasing f9[2]
def obj_rule(m):
    return m.rc['C1'] * m.f9[1] +  m.rc['C2'] * m.f9[2] \
           - (m.rc['A'] + m.vc[1]) * m.f[1] \
           - m.rc['B'] * m.f[3] \
           - m.vc[2] * m.f[5] \
           - m.vc[3] * m.f[6] \
           - sum(m.fc[i] * m.y[i] for i in m.IDXprocess)
model1.obj = pe.Objective(rule=obj_rule, sense=pe.maximize)

In [18]:
def max_demand1_rule(m):
    return m.f9[1] <= 10
model1.max_demand1 = pe.Constraint(rule=max_demand1_rule) # case 2


def max_demand2_rule(m):
    return m.f9[2] <= 5
model1.max_demand2 = pe.Constraint(rule=max_demand2_rule)


def max_supply_rule(m):
    return m.f[1] <= 16
model1.max_supply = pe.Constraint(rule=max_supply_rule)


# mass balance - chemical process
def mass_balanceI_rule(m):
    return m.f[2] == m.x[1] * m.f[1]
model1.mass_balanceI = pe.Constraint(rule=mass_balanceI_rule)


def mass_balanceII_rule(m):
    return m.f[7] == m.x[2] * m.f[5]
model1.mass_balanceII = pe.Constraint(rule=mass_balanceII_rule)


def mass_balanceIII_rule(m):
    return m.f[8] == m.x[3] * m.f[6]
model1.mass_balanceIII = pe.Constraint(rule=mass_balanceIII_rule)


# mass balance - mix and split
def mass_balanceB1_rule(m):
    return m.f[4] == m.f[5] + m.f[6]
model1.mass_balanceB1 = pe.Constraint(rule=mass_balanceB1_rule)


def mass_balanceB2_rule(m):
    return m.f[4] == m.f[2] + m.f[3]
model1.mass_balanceB2 = pe.Constraint(rule=mass_balanceB2_rule)


def mass_balanceC_rule(m):
    return m.f9[1] + m.f9[2] == m.f[7] + m.f[8]
model1.mass_balanceC = pe.Constraint(rule=mass_balanceC_rule)


def upper_bound1_rule(m):
    return m.f[1] <= 16 * m.y[1]
model1.upperbound1 = pe.Constraint(rule=upper_bound1_rule)


def upper_bound2_rule(m):
    return m.f[5] <= 15/m.x[2] * m.y[2]
model1.upperbound2 = pe.Constraint(rule=upper_bound2_rule)


def upper_bound3_rule(m):
    return m.f[6] <= 15/m.x[3] * m.y[3]
model1.upperbound3 = pe.Constraint(rule=upper_bound3_rule)


def exclusive_strategy_rule(m):
    return sum(m.y[i] for i in [2, 3]) == 1
model1.exclusive_strategy = pe.Constraint(rule=exclusive_strategy_rule)

In [19]:
# 1.2
solver1 = pe.SolverFactory('gurobi')
result_obj = solver1.solve(model1, tee=True)
model1.pprint()
print(result_obj)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-01
Read LP format model from file /var/folders/b0/z7p69zhx459bv344t3ml6by00000gp/T/tmpnhpx04ct.pyomo.lp
Reading time = 0.00 seconds
x14: 14 rows, 14 columns, 28 nonzeros
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[rosetta2])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 14 rows, 14 columns and 28 nonzeros
Model fingerprint: 0x2665916a
Variable types: 11 continuous, 3 integer (3 binary)
Coefficient statistics:
  Matrix range     [8e-01, 2e+01]
  Objective range  [4e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 9 rows and 7 columns
Presolve time: 0.00s
Presolved: 5 rows, 7 columns, 13 nonzeros
Variable types: 5 continuous, 2 integer (2 binary)
Found heuristic solution: objective 4.5934959

Root relaxation: objective 8.742690e+00, 3 iterations, 0.00 seconds (0.

In [20]:
model2 = pe.ConcreteModel()
model2.IDXt = pe.RangeSet(6)

demand = {1: 10, 2: 40, 3: 20, 4: 5, 5: 5, 6: 15}
fixed_cost = {1: 50, 2: 50, 3: 50, 4: 50, 5: 50, 6: 50}
production_cost = {1: 1, 2: 3, 3: 3, 4: 1, 5: 1, 6: 1}
holding_cost = {1: 2, 2: 2, 3: 2, 4: 2, 5: 2, 6: 2}

model2.d = pe.Param(model2.IDXt, initialize=demand)
model2.f = pe.Param(model2.IDXt, initialize=fixed_cost)
model2.p = pe.Param(model2.IDXt, initialize=production_cost)
model2.h = pe.Param(model2.IDXt, initialize=holding_cost)
model2.C = pe.Param(initialize=25)

model2.y = pe.Var(model2.IDXt, within=pe.NonNegativeReals)
model2.s = pe.Var(model2.IDXt, within=pe.NonNegativeReals)
model2.x = pe.Var(model2.IDXt, within=pe.Binary)

In [21]:
def obj_rule(m):
    return sum(m.p[t] * m.y[t] + m.h[t] * m.s[t] + m.f[t] * m.x[t] for t in m.IDXt)
model2.obj = pe.Objective(rule=obj_rule, sense=pe.minimize)

In [22]:
def balance_rule(m, t):
    if t == 1:
        return 0 + m.y[t] == m.d[t] + m.s[t]
    else:
        return m.s[t-1] + m.y[t] == m.d[t] + m.s[t]    
model2.balance = pe.Constraint(model2.IDXt, rule=balance_rule)

def production_limit_rule(m, t):
    return m.y[t] <= m.C * m.x[t]
model2.production_limit = pe.Constraint(model2.IDXt, rule=production_limit_rule)

In [23]:
# 2
solver2 = pe.SolverFactory('gurobi')
result_obj = solver2.solve(model2, tee=True)
model2.pprint()
print(result_obj)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-01
Read LP format model from file /var/folders/b0/z7p69zhx459bv344t3ml6by00000gp/T/tmpq7__50i0.pyomo.lp
Reading time = 0.00 seconds
x19: 13 rows, 19 columns, 30 nonzeros
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[rosetta2])

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 13 rows, 19 columns and 30 nonzeros
Model fingerprint: 0x7b2980b9
Variable types: 13 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Presolve removed 9 rows and 13 columns
Presolve time: 0.00s
Presolved: 4 rows, 6 columns, 9 nonzeros
Variable types: 0 continuous, 6 integer (3 binary)
Found heuristic solution: objective 485.0000000

Root relaxation: objective 4.475000e+02, 4 iterations, 0.00 seconds (